# 🥉 Camada Bronze — Openbrewery

Este notebook implementa a camada **Bronze** da arquitetura **Medallion** no Databricks utilizando:

- Auto Loader (cloudFiles)
- Unity Catalog Volumes
- Delta Lake
- Streaming incremental com checkpoint

O objetivo da Bronze é armazenar os dados **exatamente como chegam da origem**, sem qualquer transformação.


## 📥 Origem dos dados

O Databricks Community Edition não permite chamadas externas para APIs.

Por isso, os dados da Open Brewery API são coletados externamente e adicionados ao Volume do Unity Catalog: `/Volumes/bees-teste-jp/default/landing/openbrewery/`

No meu cenário extrai os dados utilizando o Google Colab através do crawler_coleta_openbrewery em anexo no repositório. Devido a limitações do Databricks Free Edition, não foi possivel consumir diretamente na bronze e nem coletar externamente e disponibilizar no volume desejado.

Código usado para coletar do dados:

```python
import requests
import json
from datetime import datetime
import time

data_hoje = datetime.now().strftime("%Y-%m-%d")
all_data = []
page = 1
max_retries = 3

print(f"🚀 Iniciando extração: {data_hoje}")

while True:
    url = f"https://api.openbrewerydb.org/v1/breweries?page={page}&per_page=200"

    success = False
    for attempt in range(max_retries):
        try:
            resp = requests.get(url, timeout=10)
            resp.raise_for_status()
            data = resp.json()
            success = True
            break

        except requests.exceptions.RequestException as e:
            print(f"⚠️ Erro na página {page} (Tentativa {attempt + 1}): {e}")
            time.sleep(2)

    if not success:
        print(f"❌ Falha crítica ao acessar a página {page}. Abortando.")
        break

    if not data:
        break

    all_data.extend(data)
    print(f"✅ Página {page} processada. Total acumulado: {len(all_data)}")
    page += 1

if all_data:
    nome_arquivo = f"breweries_raw_{data_hoje}.json"
    try:
        with open(nome_arquivo, "w") as f:
            json.dump(all_data, f)

        from google.colab import files
        files.download(nome_arquivo)
        print(f"🏁 Extração concluída! {len(all_data)} registros salvos.")
    except Exception as e:
        print(f"❌ Erro ao salvar o arquivo: {e}")
else:
    print("⚠️ Nenhum dado foi extraído.")
```



Cada execução da API gera um novo arquivo JSON nessa pasta.

Isso simula um cenário real de Data Lake, onde arquivos chegam em uma área de *landing*.



## 🚀 Por que usar Auto Loader?

Em vez de usar `spark.read.json`, utilizamos o **Auto Loader**:

`spark.readStream.format("cloudFiles")`

Vantagens:

- Processa apenas arquivos novos (incremental)
- Mantém estado via checkpoint
- Suporta evolução de schema
- Escalável para grandes volumes de arquivos

## 📖 Leitura de JSON com `multiLine=true`

O JSON da Open Brewery vem no formato de **array único**:

[
  {...},
  {...}
]

O Spark por padrão espera **um JSON por linha**.

Por isso utilizamos:

`.option("multiLine", "true")`




## 🏗️ Código de leitura da Bronze

> ⚠️ **Observação importante sobre a estratégia de carga (FULL SNAPSHOT)**

A API OpenBrewery **não disponibiliza nenhum campo de controle incremental** (como `updated_at`, `last_modified`, CDC, etc.) que permita identificar apenas os registros novos ou alterados a cada execução.

Na prática, a cada dia a API retorna **o dataset completo novamente** (snapshot do estado atual).

Por esse motivo, **não utilizamos a estratégia tradicional de append na Bronze**, pois isso faria a tabela crescer indefinidamente com dados duplicados, gerando:

- Aumento desnecessário de storage
- Necessidade de deduplicação complexa na Silver
- Maior custo computacional
- Dificuldade para identificar o “estado atual” da fonte

---

### ✅ Estratégia adotada

Optamos por tratar essa ingestão como **FULL SNAPSHOT diário**, aplicando o seguinte padrão:

1. Antes da carga, a tabela Bronze é **truncada**
2. O Auto Loader grava o snapshot completo do dia na Bronze
3. Cada carga gera uma **nova versão da tabela Delta**

Dessa forma:

- A Bronze sempre representa **o estado mais atual da fonte**
- Evitamos duplicações desnecessárias
- Mantemos o histórico de versões através do **Delta Time Travel**
- O storage permanece estável, crescendo apenas quando a fonte realmente cresce

---

### 🔎 Consulta a versões anteriores (Delta Time Travel)

Caso seja necessário consultar o estado da Bronze em dias anteriores:

```sql
SELECT * 
FROM bronze_openbrewery
VERSION AS OF <versão>;


In [0]:
%sql
USE CATALOG `bees-teste-jp`;
USE SCHEMA default;

In [0]:
%sql
TRUNCATE TABLE `bees-teste-jp`.default.bronze_openbrewery;

In [0]:
df_bronze = (
    spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "json")
    .option("multiLine", "true")
    .option("cloudFiles.schemaLocation", "/Volumes/bees-teste-jp/default/checkpoints/openbrewery_schema")
    .load("/Volumes/bees-teste-jp/default/landing/openbrewery/")
)

## 💾 Escrita na tabela Delta Bronze

Os dados são escritos em uma tabela Delta utilizando streaming.

O `checkpointLocation` é o que garante que:

- O Auto Loader saiba quais arquivos já foram ingeridos
- O streaming possa continuar de onde parou

## ⏱️ Uso do `trigger(availableNow=True)`

Esse modo executa o streaming como um **batch inteligente**:

- Processa todos os arquivos novos disponíveis
- Finaliza a execução
- Mantém o estado salvo no checkpoint

Na próxima execução, apenas novos arquivos serão lidos.


In [0]:
(df_bronze.writeStream
    .format("delta")
    .option("checkpointLocation", "/Volumes/bees-teste-jp/default/checkpoints/openbrewery_checkpoint")
    .trigger(availableNow=True)
    .toTable("`bees-teste-jp`.default.bronze_openbrewery")
)
